*Group 30 Contributers: Zhong Tan, Nicholas Low, Stephen Huang * 

*To group 30 members: add your name here as proof of contribution*

To edit, go inside Jupyter. Click the diamond icon on the left bar. Click the cloud down arrow. Edit changes. Save. The project is now under [Changed]. Hover over it and click on the [+]. Fill in the summary with what you changed. Click [COMMIT]. Click the cloud up arrow. Everything is now updated.

**Using R and knn regression to predict used car costs in the US based on mileage, brand and year**

**Introduction**

Being able to predict used car prices can save families thousands of dollars. By using the progam language R, we can plot graphs and get a prediction for how much a car may cost. We can use a method called regression and predict a numerical value for a used car's price based on factors we enter.

We will predict the cost of a used car based on mileage and year. As background knowledge, we know that newer products with more technology cost more comapred tp older products with less features. We apply this knowledge to used cars in our prediction. We also know that more used products are valued less than products in better conditions. Knowing that used or worn products are valued less is why we choose mileage as a factor. We believe that overall used car prices will be changed when we change mileage and year.

In our example, we use linear regression, as we want interpretability in our final results. In contrast, another option, KNN regression, tends to be more confusing to interpret. With linear regression, we might be able to predict the cost of a used car based on mileage and year. We will be using used, American car prices scraped from the web. We have taken this dataset and put it into our group's repository for easy access. To extract this dataset, we will start with read_csv() of our comma seperated sheet in .csv format, USA_cars_datasets.csv.

**Preliminary exploratory analysis**

We can simply read the online file by putting the raw data link into read_csv( ) in our code. But first we need to load our libraries.

In [5]:
#install.packages("tidyverse")
#install.packages("readr")
#options(repr.matrix.max.rows = 8)

In [6]:
#library(tidyverse)
#library(readr)
#library(repr)
#library(rvest)
#library(stringr)
#library(RCurl)
#library(tidymodels)
#library(gridExtra)

Okay, now let's see the data.

In [7]:
car <- read_csv("https://raw.githubusercontent.com/zhtgeneral/Group-30-Official/main/Data/USA_cars_datasets.csv")
head(car)

ERROR: Error in read_csv("https://raw.githubusercontent.com/zhtgeneral/Group-30-Official/main/Data/USA_cars_datasets.csv"): could not find function "read_csv"


It seems like our data is already in a clear format. We have 1 observation per row and 1 variable per column. We can summarize how many observations our training data will have.

In [9]:
#car_split <- initial_split(car, prop = 0.75, strata =price)
#car_training <- training(car_split)
#car_testing <- testing(car_split)

In [10]:
#eda_tibble <- tibble(nrow(car_training))
#eda_tibble

We seem to have 1872 observations. We can also find the average price of a used car based on the brand of the car, and we can sort by most abundant brands.

In [11]:
#car_avg <- car_training |>
#             group_by(brand) |>
#             summarise(count = n(),
#                       avg_price = mean(price)) |>
#             arrange(-count) 
#car_avg

We seem to have mostly Fords, Dodges, Nissans, and Chevrolets in our training data. We can now make a plot of the average price of a used car relative to the car brand and only plot the top 10 most abundant brands.


In [12]:
#car_avg_common <- car_avg |>
#                 slice(1:10)
#eda_plot <- car_avg_common |>
#            ggplot(aes(x=avg_price, y=fct_reorder(brand, count, .desc = FALSE))) +
#            geom_bar(stat="identity") +
#            labs(x="Average price of used car",
#            y="Brand of used car in order of most abundant") +
#            theme(text = element_text(size = 20)) +
#            ggtitle("Average cost of top 10 car \nbrands in the US based on total \nnumber of cars")
#
#eda_plot

We can notice that BWMs are not as abundant as Fords, but BMWs cost more. If we have a random, used BWM, we expect that it costs more.

**Methods**

To conduct our data analysis, we will look at only mileage, production year of the car, and the brand. The lot number, vin number, state, and the day the listing was uploaded doesn't make a difference. We can visualize our results with many scatterplots. We can use a scatterplot with price on the y axis and year/mileage on the x axis then have a bar plot with price on the x axis and brand on the y axis (like the graph above). We can also get a  measure of accuracy if we use a model called knn regression.

*KNN regression vs linear regression*

*Use GGPairs*

**Expected Outcomes and significance**

We can expect used BMWs and Fords to cost more than the average used car. We can expect the cost of a random used car to be around between 6000 and $22000 USD based on the top 10 most abundant used cars. Being able to estimate the cost of a used car based on brand, mileage, and year can give us knowledge to know whether to wait for a better deal or to buy now. This analysis can bring insight to the question of how much can designer brand cars with high mileage be cheaper than economy brand cars with low mileage.

*Make sure to relate to intro*
